<a href="https://colab.research.google.com/github/kameshcodes/deep-learning-codes/blob/main/Lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-summary --quiet
!pip install torchvision --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torchsummary import summary

In [3]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,
                           kernel_size=5, stride=1, padding=0)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16,
                           kernel_size=5, stride=1, padding=0)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120,
                           kernel_size=5, stride=1, padding=0)
    self.linear1 = nn.Linear(in_features=120, out_features=84)
    self.linear2 = nn.Linear(in_features=84, out_features=10)
    self.tanh = nn.Tanh()
    self.avgpool = nn.AvgPool2d(kernel_size=2, stride = 2)

  def forward(self, x):
    x = self.conv1(x)
    x = self.tanh(x)
    x = self.avgpool(x)
    x = self.conv2(x)
    x = self.tanh(x)
    x = self.avgpool(x)
    x = self.conv3(x)
    x = self.tanh(x)
    x = x.reshape(x.shape[0], -1)
    x = self.linear1(x)
    x = self.tanh(x)
    x = self.linear2(x)
    return x

In [4]:
model = LeNet()

summary(model, (1, 32, 32), verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 28, 28]           156
├─Tanh: 1-2                              [-1, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [-1, 6, 14, 14]           --
├─Conv2d: 1-4                            [-1, 16, 10, 10]          2,416
├─Tanh: 1-5                              [-1, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [-1, 16, 5, 5]            --
├─Conv2d: 1-7                            [-1, 120, 1, 1]           48,120
├─Tanh: 1-8                              [-1, 120, 1, 1]           --
├─Linear: 1-9                            [-1, 84]                  10,164
├─Tanh: 1-10                             [-1, 84]                  --
├─Linear: 1-11                           [-1, 10]                  850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
Total mult-adds (M): 0.42
Input size (MB): 0.00
Forward/backward pass si

# Data Loading

In [5]:
learning_rate = 0.01
num_epochs = 5

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.Compose([
                                   transforms.Pad(2),  # Padding each image with 2 pixels
                                   transforms.ToTensor()  # Converting to PyTorch tensor
                               ]),
                               download=True)

test_dataset = datasets.MNIST(root='dataset/', train=False,
                              transform=transforms.Compose([
                                  transforms.Pad(2),  # Padding each image with 2 pixels
                                  transforms.ToTensor()  # Converting to PyTorch tensor
                              ]))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 14522556.21it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 497281.13it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1447151.64it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5458604.23it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size = 64, shuffle = True)

In [9]:
dataset_sizes = {'train':len(train_dataset), 'test':len(test_dataset)}
dataset_sizes

{'train': 60000, 'test': 10000}

In [10]:
model

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)

In [11]:
model = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
import time

def train_Lenet(model, criterion, optimizer, num_epochs, dataloaders, dataset_sizes, first_epoch=1):
  since = time.time()
  best_loss = float('inf')
  best_epoch = -1
  last_train_loss = -1
  plot_train_loss = []
  plot_valid_loss = []

  for epoch in range(first_epoch, first_epoch + num_epochs):
    print()
    print('Epoch', epoch)
    running_loss = 0.0
    valid_loss = 0.0

    model.train()

    for data in dataloaders[0]:
        inputs, labels  = data
        batch_size = inputs.shape[0]

        #backprop
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.data * batch_size

    epoch_loss = running_loss / dataset_sizes['train']
    plot_train_loss.append(epoch_loss)
    print('Train Loss: {:.4f}'.format(epoch_loss))


    model.eval()
    with torch.no_grad():
      for data in dataloaders[-1]:
        input, labels = data
        batch_size = inputs.shape[0]

        output = model(input)
        loss = criterion(output, labels)
        valid_loss += loss.data * batch_size

    epoch_loss = valid_loss / dataset_sizes['test']
    plot_valid_loss.append(epoch_loss)
    print('Valid Loss: {:.4f}'.format(epoch_loss))

  time_elapsed  = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  return plot_train_loss, plot_valid_loss, model

In [22]:
train_losses, valid_losses, model = train_Lenet(model = model ,criterion = criterion,optimizer = optimizer,
                                          num_epochs=10, dataloaders = [train_loader, test_loader],
                                          dataset_sizes = dataset_sizes)


Epoch 1
Train Loss: 0.1055
Valid Loss: 0.0503

Epoch 2
Train Loss: 0.1090
Valid Loss: 0.0645

Epoch 3
Train Loss: 0.1122
Valid Loss: 0.0508

Epoch 4
Train Loss: 0.1129
Valid Loss: 0.0584

Epoch 5
Train Loss: 0.1247
Valid Loss: 0.0582

Epoch 6
Train Loss: 0.1307
Valid Loss: 0.0588

Epoch 7
Train Loss: 0.1133
Valid Loss: 0.0541

Epoch 8
Train Loss: 0.1272
Valid Loss: 0.0644

Epoch 9
Train Loss: 0.1097
Valid Loss: 0.0531

Epoch 10
Train Loss: 0.1078
Valid Loss: 0.0576
Training complete in 5m 45s


In [24]:
def accuracy(loader, model, train=True):
    num_correct = num_samples = 0
    model.eval()
    with torch.no_grad():
      for data in loader:
        inputs, labels  = data
        batch_size = inputs.shape[0]

        outputs = model(inputs)
        _, preds = outputs.max(1)
        num_correct += (preds == labels).sum()
        num_samples += preds.size(0)
    accuracy = (num_correct.item()/num_samples)*100
    if train:
      print(f"Model Predicted {num_correct} correctly out of {num_samples} from training dataset, Accuracy: {accuracy:.2f}")
    else:
      print(f"Model Predicted {num_correct.item()} correctly out of {num_samples} from testing dataset, Accuracy: {accuracy:.2f}")
    model.train()

accuracy(train_loader, model)
accuracy(test_loader, model, train=False)

Model Predicted 58014 correctly out of 60000 from training dataset, Accuracy: 96.69
Model Predicted 9660 correctly out of 10000 from testing dataset, Accuracy: 96.60


In [23]:
import plotly.express as px

epochs = list(range(1, len(train_losses) + 1))

fig = px.line(x=epochs, y=train_losses, labels={'x': 'Epoch', 'y': 'Loss'},
              title='Training Loss Over Epochs')
fig.add_scatter(x=epochs, y=valid_losses, mode='lines', name='Validation Loss')

fig.show()